# Защита персональных данных клиентов

Необходимо защитить данные, разработав метод преобразования данных, который бы усложнил восстановление персональных данных. При этом нужно, чтобы качество моделей машинного обучения для зашифрованных данных не ухудшилось.

## Загрузка данных

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression

In [2]:
df = pd.read_csv('datasets/insurance.csv')

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Пол                5000 non-null   int64  
 1   Возраст            5000 non-null   float64
 2   Зарплата           5000 non-null   float64
 3   Члены семьи        5000 non-null   int64  
 4   Страховые выплаты  5000 non-null   int64  
dtypes: float64(2), int64(3)
memory usage: 195.4 KB


In [4]:
df.head()

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
0,1,41.0,49600.0,1,0
1,0,46.0,38000.0,1,1
2,0,29.0,21000.0,0,0
3,0,21.0,41700.0,2,0
4,1,28.0,26100.0,0,0


## Умножение матриц

### Вопрос

Обозначения:

- $X$ — матрица признаков (нулевой столбец состоит из единиц)

- $y$ — вектор целевого признака

- $P$ — матрица, на которую умножаются признаки

- $w$ — вектор весов линейной регрессии (нулевой элемент равен сдвигу)

Предсказания:

$$
a = Xw
$$

Задача обучения:

$$
w = \arg\min_w MSE(Xw, y)
$$

Формула обучения:

$$
w = (X^T X)^{-1} X^T y
$$

**Признаки умножают на обратимую матрицу. Изменится ли качество линейной регрессии?**

### Ответ

Не изменится.

### Обоснование

Предположим, что мы преобразовали матрицу признаков $X$, умножив её на некую обратимую матрицу $P$. 

$$
X_P = XP
$$

Тогда изменённый вектор весов $w_P$ можно представить так:

$$
w_P = (X_P^T X_P)^{-1} X_P^T y
$$

$$
w_P = ((XP)^T(XP))^{-1} (XP)^Ty
$$

Воспользуемся следующими свойствами умножения матриц:
$$
(A^T)^{−1}=(A^{−1})^T.
$$
$$
(AB)^{−1}=B^{−1}A^{−1}
$$
$$
(AB)^T=B^TA^T
$$
$$
(AB)C=A(BC)
$$

Итак:

$$
w_P = ((XP)^T(XP))^{-1} (XP)^Ty
$$

$$
w_P = (XP)^{-1}((XP)^T)^{-1}(XP)^Ty
$$

$$
w_P = P^{-1}X^{-1}((XP)^T)^{-1}(XP)^Ty
$$

$$
w_P = P^{-1}((XP)^TX)^{-1}(XP)^Ty
$$

$$
w_P = P^{-1}(P^TX^TX)^{-1}(XP)^Ty
$$

$$
w_P = P^{-1}(X^TX)^{-1}(P^T)^{-1}(XP)^Ty
$$

$$
w_P = P^{-1}(X^TX)^{-1}(P^T)^{-1}P^TX^Ty
$$

$$
w_P = P^{-1}(X^TX)^{-1}X^Ty
$$


$$
w_P = P^{-1}w
$$

Таким образом, если мы обучим модель преобразованными данными $X_P$, получим изменённый вектор весов $w_P$ и будем проводить валидацию на преобразованных данных $Z_P = ZP$, то получим следующие предсказания:


$$
a_P = Z_Pw_P
$$

$$
a_P = ZPP^{-1}w
$$

$$
a_P = Zw
$$

Мы видим, что предсказания будут соответствовать непреобразованным данным, следовательно оценка не поменяется.

## Алгоритм преобразования

### Алгоритм

Умножение матриц разных размеров


$$
A_{m × n} B_{n × p} = C_{m × p}
$$

Чтобы получить матрицу $C$ такого же размера как и $A$, матрица $B$ должна быть квадратной. 

$$
A_{m × n} B_{n × n} = C_{m × n}
$$

1. Сгенерируем квадратную матрицу-"шифр" размера количества признаков (4)
2. Убедимся, что она обратимая
3. Подготовим обучающую и валидационную выборки
4. Обучим модель на обучающей выборке
5. Рассчитаем качество модели на валидационной
6. Преобразуем обучающую и валидационную выборки, умножив их на матрицу-"шифр"
7. Снова обучим модель 
8. Рассчитаем качество модели для зашифрованных данных
9. Проверим совпадают ли оценки качества

### Обоснование

Согласно обоснованию в пункте 2.3 при обучении и валидации на зашифрованных данных мы должны получить незашифрованные предсказания. Исходя из этого метрика должна совпасть с метрикой для незашифрованных данных.

## Проверка алгоритма

### Сгенерируем квадратную матрицу-"шифр"

In [5]:
## генерирует матрицу с проверкой на обратимость
def get_inv_matrix(loc, scale, size):
    np.random.seed(42)
    success = False
    M = None
    while not success:
        M = np.random.normal(loc, scale, size=size)
        M_inv = np.linalg.inv(M)
        success = np.allclose(np.dot(M, M_inv), np.eye(M.shape[0]))
    return M

In [6]:
P_size = (df.shape[1]-1, df.shape[1]-1)

In [7]:
P = get_inv_matrix(50, 25, P_size)

In [8]:
P

array([[62.41785383, 46.54339247, 66.19221345, 88.07574641],
       [44.14616563, 44.14657608, 89.48032039, 69.18586823],
       [38.26314035, 63.56400109, 38.41455768, 38.35675616],
       [56.04905679,  2.16799388,  6.87705419, 35.94281177]])

### Подготовим обучающую и валидационную выборки

In [9]:
target = df['Страховые выплаты']
features = df.drop('Страховые выплаты', axis=1)

In [10]:
features_train, features_test, target_train, target_test = train_test_split(
    features, target, test_size=0.4, random_state=42)

### Обучим модель на обучающей выборке

In [11]:
scaler = StandardScaler()
scaler.fit(features_train)
features_train = scaler.transform(features_train)
features_train

array([[-0.99534417,  0.82549801, -0.23442471, -0.17941693],
       [-0.99534417,  0.10857414,  1.0440908 , -0.17941693],
       [ 1.00467761,  1.18395994,  0.32933016, -0.17941693],
       ...,
       [ 1.00467761,  0.82549801,  0.66154285, -1.07799923],
       [ 1.00467761,  0.22806145, -0.40556458, -1.07799923],
       [-0.99534417,  1.66190919,  0.44006772,  0.71916537]])

In [12]:
features_test = scaler.transform(features_test)
features_test

array([[ 1.00467761, -0.36937511,  1.62797977, -1.07799923],
       [ 1.00467761,  0.10857414,  0.19845849,  0.71916537],
       [ 1.00467761, -0.13040049, -1.3720015 , -1.07799923],
       ...,
       [ 1.00467761,  2.85678231, -0.68744202, -0.17941693],
       [-0.99534417, -1.44476092,  0.2387267 , -0.17941693],
       [ 1.00467761,  3.33473155,  1.18502951,  0.71916537]])

In [13]:
model = LinearRegression()

In [14]:
model.fit(features_train, target_train)

LinearRegression()

### Рассчитаем качество модели на валидационной

In [15]:
predicted_test = model.predict(features_test)

In [16]:
predicted_test

array([ 0.05029542,  0.16391432,  0.13466629, ...,  0.98989569,
       -0.27411194,  1.10433064])

In [17]:
r2 = model.score(features_test, target_test)

In [18]:
r2

0.4268018208826947

### Преобразуем обучающую и валидационную выборки, умножив их на матрицу-"шифр"

In [19]:
features_train_p = features_train.dot(P)

In [20]:
features_train_p

array([[-44.71065065, -25.1737312 ,  -2.25738919, -45.99340495],
       [-27.44007233,  24.44409563, -17.29435916, -46.55469786],
       [117.52216798, 119.57344956, 183.86016199, 176.58431425],
       ...,
       [ 64.0442586 , 122.91722936, 158.36707759, 132.22884083],
       [ -3.16115604,  28.71283302,  63.91580299,  49.96389431],
       [ 68.38658475,  56.57251731, 104.67487895,  70.04334501]])

In [21]:
features_test_p = features_test.dot(P)

In [22]:
features_test_p

array([[  48.27410349,  131.59826969,   88.57469496,   86.62989199],
       [ 115.40513738,   65.72824249,   88.78651755,  129.4605754 ],
       [ -55.96478743,  -48.54263137,   -5.28473255,  -11.90599111],
       ...,
       [ 152.46596474,  128.79272174,  294.48598976,  253.31989956],
       [-126.82961836,  -95.32249285, -187.22498293, -184.91468646],
       [ 295.57692247,  270.86244773,  415.36280619,  390.50674138]])

### Снова обучим модель

In [23]:
features_train_p = features_train.dot(P)

In [24]:
scaler_p = StandardScaler()
scaler_p.fit(features_train_p)
features_train_p = scaler_p.transform(features_train_p)
features_train_p

array([[-0.44154607, -0.27744764, -0.0190751 , -0.3721098 ],
       [-0.27098814,  0.26940609, -0.14613858, -0.37665095],
       [ 1.16060603,  1.31785673,  1.55363165,  1.42865601],
       ...,
       [ 0.63247772,  1.35470958,  1.33821324,  1.0697979 ],
       [-0.03121842,  0.3164532 ,  0.54009315,  0.40423306],
       [ 0.67536095,  0.62350357,  0.88451029,  0.56668593]])

In [25]:
features_test_p = scaler_p.transform(features_test_p)
features_test_p

array([[ 0.47673743,  1.45038607,  0.74846257,  0.70087945],
       [ 1.13969901,  0.72441171,  0.75025249,  1.04740124],
       [-0.55268781, -0.53500366, -0.04465637, -0.09632546],
       ...,
       [ 1.50569821,  1.41946524,  2.48842788,  2.04948554],
       [-1.25252302, -1.05057928, -1.58206463, -1.49605292],
       [ 2.91900979,  2.98526054,  3.50984571,  3.15939616]])

In [26]:
model_p = LinearRegression()

In [27]:
model_p.fit(features_train_p, target_train)

LinearRegression()

### Рассчитаем качество модели на валидационной

In [28]:
predicted_test_p = model_p.predict(features_test_p)

In [29]:
predicted_test_p

array([ 0.05029542,  0.16391432,  0.13466629, ...,  0.98989569,
       -0.27411194,  1.10433064])

In [30]:
r2_p = model_p.score(features_test_p, target_test)

In [31]:
r2_p

0.4268018208826947

In [32]:
r2

0.4268018208826947

### Вывод

Мы получили две оценки r2, которые не отличаются. Это подтверждает наше предположение, что можно строить модели и расчитывать метрики даже с "зашифрованными" данными методом умножения на обратимую матрицу.